# Effect of repealing charitable deduction

This identifies beneficiaries of the charitable deduction by modeling its repeal. Both repeal from current (2017) state and TCJA state are considered on a static basis. Change to after-tax income by decile and share of after-tax income held by top 10% are calculated.

*taxcalc version: 0.14.3  |  Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2017-12-30*

## Setup

### Imports

In [1]:
from __future__ import print_function  # Necessary only if using Python 2.7.
import taxcalc as tc
import pandas as pd
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

### Load reforms

Load from Github.

In [2]:
# Folders where reforms live.
GITHUB_BASE_URL = 'https://raw.githubusercontent.com/'

TAXCALC_GITHUB_BASE_URL = (GITHUB_BASE_URL +
                           'open-source-economics/Tax-Calculator/master/' +
                           'taxcalc/reforms/')

def read_url(url):
    return url_lib.urlopen(url).read()

def read_reform_github(reform_name):
    return read_url(GITHUB_BASE_URL + reform_name + '.json')

def read_reform_taxcalc_github(reform_name):
    return read_url(TAXCALC_GITHUB_BASE_URL + reform_name + '.json')

def create_static_params_github(reform_name):
    return tc.Calculator.read_json_param_objects(
        read_reform_github(reform_name), None)

def create_static_params_taxcalc_github(reform_name):
    return tc.Calculator.read_json_param_objects(
        read_reform_taxcalc_github(reform_name), None)

In [3]:
no_charitable_reform = create_static_params_github(
    'MaxGhenis/taxcalc-notebooks/master/charitable_repeal/NoCharitableDeduction')
tcja_reform = create_static_params_taxcalc_github(
    'TCJA_Reconciliation')
tcja_no_charitable_reform = create_static_params_github(
    'MaxGhenis/taxcalc-notebooks/master/charitable_repeal/TCJA_NoCharitable')

### Specify `Policy` objects for static analyses

In [4]:
recs = tc.Records.cps_constructor()

In [5]:
def static_baseline_calc(year):
    calc = tc.Calculator(records=recs, policy=tc.Policy())
    calc.advance_to_year(year)
    calc.calc_all()
    return calc

In [6]:
def static_calc_from_reform(reform, year):
    pol = tc.Policy()
    pol.implement_reform(reform['policy'])
    if pol.reform_errors:
        print(pol.reform_errors)
    calc = tc.Calculator(records=recs, policy=pol)
    calc.advance_to_year(year)
    calc.calc_all()
    # Needs more if adding behavior.
    return calc

In [7]:
baseline_calc = static_baseline_calc(2018)
no_charitable_calc = static_calc_from_reform(no_charitable_reform, 2018)
tcja_calc = static_calc_from_reform(tcja_reform, 2018)
tcja_no_charitable_calc = static_calc_from_reform(tcja_no_charitable_reform, 2018)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


## Analysis

### Change to aggregate individual income tax revenue

In [8]:
print ('Pre-TCJA: ${:0.1f}B'.
       format((no_charitable_calc.weighted_total('iitax') - 
               baseline_calc.weighted_total('iitax')) / 1e9))
print ('TCJA: ${:0.1f}B'.
       format((tcja_no_charitable_calc.weighted_total('iitax') - 
               tcja_calc.weighted_total('iitax')) / 1e9))

Pre-TCJA: $44.8B
TCJA: $22.4B


### Construct difference tables by income decile

Ignore errors ([issue](https://github.com/open-source-economics/Tax-Calculator/issues/1799)).

In [9]:
baseline_diff_table = baseline_calc.difference_table(no_charitable_calc, tax_to_diff='iitax')
tcja_diff_table = tcja_calc.difference_table(tcja_no_charitable_calc, tax_to_diff='iitax')

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


In [10]:
baseline_diff_table

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58",0.00,0.00,500.80,0.00,0.00,"43,611.20",0.00,-0.00,0.00
1,"16,988,285.70",0.00,0.00,"50,187.09",0.30,0.28,"4,760,463.72",0.01,0.00,-0.00
2,"16,990,004.05",0.00,0.00,"691,813.67",4.07,5.15,"87,448,642.66",0.20,0.03,-0.03
3,"16,989,656.05",0.00,0.00,"1,903,067.48",11.20,27.14,"461,137,528.32",1.03,0.11,-0.11
4,"16,989,954.66",0.00,0.00,"2,738,159.87",16.12,64.25,"1,091,628,802.00",2.44,0.21,-0.21
5,"16,989,240.80",0.00,0.00,"3,682,693.42",21.68,119.80,"2,035,392,083.49",4.55,0.31,-0.31
6,"16,989,686.38",0.00,0.00,"4,935,193.47",29.05,254.36,"4,321,435,690.84",9.66,0.50,-0.50
7,"16,989,484.52",0.00,0.00,"5,809,046.03",34.19,399.90,"6,794,116,838.92",15.18,0.59,-0.59
8,"16,989,613.44",0.00,0.00,"7,050,328.08",41.50,626.33,"10,641,103,380.35",23.78,0.65,-0.65
9,"16,989,497.48",0.00,0.00,"8,284,178.18",48.76,"1,136.78","19,313,295,549.07",43.16,0.54,-0.54


In [11]:
tcja_diff_table

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58",0.00,0.00,500.80,0.00,0.00,"62,591.29",0.00,-0.00,0.00
1,"16,988,285.70",0.00,0.00,"5,207.10",0.03,0.01,"222,332.56",0.00,0.00,-0.00
2,"16,990,004.05",0.00,0.00,"133,144.85",0.78,1.41,"23,923,356.00",0.11,0.01,-0.01
3,"16,989,656.05",0.00,0.00,"332,876.29",1.96,5.39,"91,611,714.09",0.41,0.02,-0.02
4,"16,989,954.66",0.00,0.00,"615,923.46",3.63,13.49,"229,159,421.60",1.02,0.04,-0.04
5,"16,989,240.80",0.00,0.00,"960,374.81",5.65,29.21,"496,188,132.72",2.21,0.07,-0.07
6,"16,989,686.38",0.00,0.00,"1,536,497.48",9.04,76.15,"1,293,752,014.36",5.77,0.15,-0.15
7,"16,989,484.52",0.00,0.00,"2,781,233.05",16.37,128.86,"2,189,182,040.94",9.76,0.19,-0.19
8,"16,989,613.44",0.00,0.00,"5,451,774.32",32.09,308.49,"5,241,064,357.63",23.37,0.32,-0.32
9,"16,989,497.48",0.00,0.00,"7,132,252.57",41.98,757.16,"12,863,684,595.64",57.35,0.35,-0.35


Focus on % change to after-tax income.

In [12]:
pd.DataFrame({'baseline': -baseline_diff_table['pc_aftertaxinc'],
              'tcja': -tcja_diff_table['pc_aftertaxinc']})

,baseline,tcja
0,-0.00,-0.00
1,0.00,0.00
2,0.03,0.01
3,0.11,0.02
4,0.21,0.04
5,0.31,0.07
6,0.50,0.15
7,0.59,0.19
8,0.65,0.32
9,0.54,0.35


Graphed in Google Sheets [here](https://docs.google.com/spreadsheets/d/17ozKKuWUzgI4yn1OwqshmPBVWVklQ1GlpTAUepuwsF8/edit?usp=sharing).

*TODO: Make a Seaborn or Bokeh plot here instead.*

### After-tax income held by top decile

Ignore errors ([issue](https://github.com/open-source-economics/Tax-Calculator/issues/1799)).

In [13]:
baseline_dist, no_charitable_dist = baseline_calc.distribution_tables(no_charitable_calc)
tcja_dist, tcja_no_charitable_dist = tcja_calc.distribution_tables(tcja_no_charitable_calc)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


In [14]:
def after_tax_income_top10pct(calc):
    return calc.aftertax_income[9] / calc.aftertax_income[10]

In [15]:
print ('After-tax income from top 10% - Baseline: {:0.3f}%'.
       format(after_tax_income_top10pct(baseline_dist) * 100))
print ('After-tax income from top 10% - No charitable: {:0.3f}%'.
       format(after_tax_income_top10pct(no_charitable_dist) * 100))
print ('After-tax income from top 10% - Difference: {:0.3f}%'.
       format((after_tax_income_top10pct(no_charitable_dist) - 
               after_tax_income_top10pct(baseline_dist)) * 100))

After-tax income from top 10% - Baseline: 38.773%
After-tax income from top 10% - No charitable: 38.751%
After-tax income from top 10% - Difference: -0.021%


In [16]:
print ('After-tax income from top 10% - Baseline: {:0.3f}%'.
       format(after_tax_income_top10pct(tcja_dist) * 100))
print ('After-tax income from top 10% - No charitable: {:0.3f}%'.
       format(after_tax_income_top10pct(tcja_no_charitable_dist) * 100))
print ('After-tax income from top 10% - Difference: {:0.3f}%'.
       format((after_tax_income_top10pct(tcja_no_charitable_dist) - 
               after_tax_income_top10pct(tcja_dist)) * 100))

After-tax income from top 10% - Baseline: 38.966%
After-tax income from top 10% - No charitable: 38.922%
After-tax income from top 10% - Difference: -0.044%
